# Week Three Assignment - Question 1

## Steps followed in creating the required dataframe

* Install all the libraries
* Download all the modules
* Read data from the Wikipedia Page
* Creating BeautifulSoup object 
* Reading and coverting BeautifulSoup oject data into dataframe
* Cleaning the data frame to remove rows with no Borough name
* Concatenating Neighborhood names which have the same Postal Code
* Adding Neighborhood names for rows missing the same
* Verify final output is corrrect and also print the shape of the final dataframe

In [ ]:
# intalling beaitifulsoup library
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge requests --yes

In [38]:
# importing necessary modules
import requests as req
import pandas as pd
import lxml
from bs4 import BeautifulSoup

In [39]:
# url for the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = req.get(url).text

In [40]:
# cretaing a beautifulsoup obect
soup = BeautifulSoup(source,'lxml')

In [44]:
# creating & loading data into dataframe
pcode_df = pd.DataFrame()
i = 0
j = 0
table_pcode = soup.find('table')
for table_data in table_pcode.find_all('td'):
    col_data = table_data.text
    pcode_df.loc[i,j] = col_data
    j = j + 1
    if (j==3):
        i = i + 1
        j = 0

In [45]:
# adding column names to the data frame
pcode_df.columns = ('PostalCode','Borough','Neighborhood')

In [46]:
# removing the \n haracter from the end of the string
pcode_df['Neighborhood'] = pcode_df['Neighborhood'].str.slice(stop = -1)
pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [47]:
# removing the rows with borough value as not assigned
pcode_df=pcode_df[pcode_df.Borough != 'Not assigned']
pcode_df = pcode_df.reset_index(drop=True)
pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [48]:
# concatenating neighborhoods with same Postal Code
new_df = pd.DataFrame()
new_df = pcode_df.groupby('PostalCode').agg({'Neighborhood':lambda x: tuple(x)})
new_df.reset_index(inplace=True)
new_df.head()

,PostalCode,Neighborhood
0,M1B,"(Rouge, Malvern)"
1,M1C,"(Highland Creek, Rouge Hill, Port Union)"
2,M1E,"(Guildwood, Morningside, West Hill)"
3,M1G,"(Woburn,)"
4,M1H,"(Cedarbrae,)"


In [49]:
# converting the tuple into a string data type
new_df['Neighborhood'] = new_df['Neighborhood'].astype(str)
new_df.head()

,PostalCode,Neighborhood
0,M1B,"('Rouge', 'Malvern')"
1,M1C,"('Highland Creek', 'Rouge Hill', 'Port Union')"
2,M1E,"('Guildwood', 'Morningside', 'West Hill')"
3,M1G,"('Woburn',)"
4,M1H,"('Cedarbrae',)"


In [50]:
#removing the end and begenning braces and the single quotes
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(stop = -2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.slice(start = 2)
new_df['Neighborhood'] = new_df['Neighborhood'].str.replace("'","")
new_df.head()

,PostalCode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [51]:
# creating a data frame of unique postal code and borough combination
df_borough = pcode_df
df_borough = df_borough.drop('Neighborhood',axis=1)
df_borough.drop_duplicates(inplace=True)
df_borough.head()

,PostalCode,Borough
0,M3A,North York
1,M4A,North York
2,M5A,Downtown Toronto
4,M6A,North York
6,M7A,Queen's Park


In [52]:
# creation of the final data frame by combining two intermediate data frames using Postal code as key
final_pcode_df = pd.DataFrame()
final_pcode_df = df_borough.merge(new_df, on='PostalCode')
final_pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [53]:
# updating the data for rows missing Neighborhood values
loc_Neigh_NA = final_pcode_df.Neighborhood == 'Not assigned'
final_pcode_df.loc[loc_Neigh_NA,'Neighborhood'] = final_pcode_df.loc[loc_Neigh_NA,'Borough']
final_pcode_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [55]:
#shaep of the final dataframe
print('Shape of the Final DataFrame',final_pcode_df.shape)

Shape of the Final DataFrame (103, 3)
